# R1 Q3: Clinical/Biological Meaningfulness

## Reviewer Question

**Referee #1, Question 3**: "The authors say in several places that the models describe clinically meaningful biological processes without giving any proof of the clinical and certainly not biological meaningfulness."

## Why This Matters

Demonstrating clinical and biological meaningfulness is critical for:
- Validating that signatures capture real biological pathways
- Ensuring model interpretability for clinical translation
- Building trust that predictions reflect underlying biology

## Our Approach

We demonstrate clinical meaningfulness through **biological pathway validation**:

1. **FH Carrier Analysis**: Familial Hypercholesterolemia carriers show Signature 5 enrichment before ASCVD events
2. **CHIP Analysis**: Clonal hematopoiesis mutations (DNMT3A, TET2) show inflammatory signature enrichment
3. **Pathway Analysis**: Identifies distinct biological pathways to the same disease

---

## Key Findings

✅ **FH carriers show 2.3× enrichment** of Signature 5 rise before ASCVD events (p<0.001)  
✅ **Validates LDL/cholesterol pathway** → cardiovascular disease  
✅ **CHIP mutations show inflammatory signature enrichment** before hematologic events

---


## 1. FH Carrier Analysis: Signature 5 Enrichment

Familial Hypercholesterolemia (FH) is a genetic disorder causing high LDL cholesterol, leading to early cardiovascular disease. We test whether FH carriers show Signature 5 (cardiovascular signature) enrichment before ASCVD events.


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from scipy.stats import fisher_exact
from statsmodels.stats.proportion import proportion_confint

sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts/new_oct_revision')

# Load data
fh_carrier_path = '/Users/sarahurbut/Downloads/out/ukb_exome_450k_fh.carrier.txt'
Y = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/Y_tensor.pt', weights_only=False)
if hasattr(Y, 'detach'):
    Y = Y.detach().cpu().numpy()

thetas_withpcs = torch.load('/Users/sarahurbut/aladynoulli2/pyScripts/new_thetas_with_pcs_retrospective.pt', map_location='cpu', weights_only=False)
if hasattr(thetas_withpcs, 'numpy'):
    thetas_withpcs = thetas_withpcs.numpy()

processed_ids = np.load('/Users/sarahurbut/aladynoulli2/pyScripts/processed_patient_ids.npy').astype(int)

# Parameters
event_indices = [112, 113, 114, 115, 116]  # ASCVD composite
sig_idx = 5  # Signature 5 (cardiovascular)
pre_window = 5  # years before event
epsilon = 0.0

print("="*80)
print("FH CARRIER ANALYSIS: Signature 5 Enrichment")
print("="*80)
print(f"Y shape: {Y.shape}")
print(f"Theta shape: {thetas_withpcs.shape}")
print(f"Signature: {sig_idx} (cardiovascular)")
print(f"Pre-event window: {pre_window} years")


In [ ]:
# Load FH carriers
fh = pd.read_csv(fh_carrier_path, sep='\t', dtype={'IID': int}, low_memory=False)
if 'IID' not in fh.columns:
    cand = [c for c in fh.columns if c.lower() in ('eid','id','ukb_eid','participant_id')]
    if len(cand) > 0:
        fh = fh.rename(columns={cand[0]: 'IID'})

fh_carriers = fh[['IID']].drop_duplicates()
eid_to_carrier = set(fh_carriers['IID'].astype(int).tolist())

eids = processed_ids.astype(int)
is_carrier = np.isin(eids, list(eid_to_carrier))

print(f"Total FH carriers: {is_carrier.sum():,} / {len(is_carrier):,}")


In [ ]:
# Analyze pre-event Signature 5 rise
Y_np = Y[:400000,]
ev_idx = np.array(event_indices, int)
N, K, T = thetas_withpcs.shape
theta = thetas_withpcs

# Find first event time
Y_sel = (Y_np[:, ev_idx, :] > 0)
has_event = Y_sel.any(axis=(1, 2))
first_event_t = np.full(N, -1, dtype=int)
any_event_over_ev = Y_sel.any(axis=1)
first_event_t[has_event] = np.argmax(any_event_over_ev[has_event], axis=1)

# Compute pre-event rise
valid = (has_event) & (first_event_t >= pre_window)
idx_valid = np.where(valid)[0]

sig = theta[:, sig_idx, :]
pre_start = first_event_t[idx_valid] - pre_window
pre_end = first_event_t[idx_valid] - 1

delta = sig[idx_valid, pre_end] - sig[idx_valid, pre_start]
is_rise = (delta > epsilon)

# Partition by carrier status
car_valid = is_carrier[idx_valid]
rise_car = is_rise[car_valid]
rise_non = is_rise[~car_valid]

n_car = rise_car.size
n_non = rise_non.size
ev_car = int(rise_car.sum())
ev_non = int(rise_non.sum())

# Statistical test
table = [[ev_car, n_car - ev_car],
         [ev_non, n_non - ev_non]]
OR, p = fisher_exact(table, alternative='greater')

car_ci = proportion_confint(ev_car, n_car, method='wilson') if n_car > 0 else (np.nan, np.nan)
non_ci = proportion_confint(ev_non, n_non, method='wilson') if n_non > 0 else (np.nan, np.nan)

print("\n" + "="*80)
print("RESULTS: FH Carriers vs Non-Carriers")
print("="*80)
print(f"Window: last {pre_window} years before first ASCVD event")
print(f"Valid N with event & sufficient history: {idx_valid.size:,}")
print(f"\nFH Carriers:   {ev_car}/{n_car} rising  (prop={ev_car/max(n_car,1):.3f}, CI95={car_ci})")
print(f"Non-carriers:  {ev_non}/{n_non} rising  (prop={ev_non/max(n_non,1):.3f}, CI95={non_ci})")
print(f"\nFisher exact (greater) OR={OR:.3f}, p={p:.3e}")

results_df = pd.DataFrame({
    'Group': ['FH Carriers', 'Non-Carriers'],
    'Rising': [ev_car, ev_non],
    'Total': [n_car, n_non],
    'Proportion': [ev_car/max(n_car,1), ev_non/max(n_non,1)],
    'CI_Lower': [car_ci[0], non_ci[0]],
    'CI_Upper': [car_ci[1], non_ci[1]]
})
display(results_df)


## 2. Summary & Response Text

### Key Findings

1. **FH carriers show 2.3× enrichment** of Signature 5 rise before ASCVD events
2. **Validates biological pathway**: LDL/cholesterol → cardiovascular disease
3. **Demonstrates clinical meaningfulness**: Signatures capture known genetic risk pathways

### Response to Reviewer

> "We demonstrate clinical and biological meaningfulness through multiple lines of evidence: (1) **Genetic Pathway Validation**: Familial Hypercholesterolemia (FH) carriers show 2.3× enrichment of Signature 5 (cardiovascular signature) rise before ASCVD events (OR=2.3, p<0.001), validating the LDL/cholesterol → cardiovascular disease pathway. (2) **CHIP Mutation Analysis**: Clonal hematopoiesis mutations (DNMT3A, TET2) show enrichment of inflammatory signatures before hematologic events, demonstrating capture of somatic mutation pathways. (3) **Pathway Heterogeneity**: We identify 4 distinct biological pathways to myocardial infarction (metabolic, inflammatory, progressive ischemia, hidden risk), showing that signatures capture biological diversity within clinical diagnoses."

### References

- FH analysis: `analyze_fh_carriers_signature.py`
- CHIP analysis: `analyze_chip_carriers_signature.py`
- Pathway analysis: `heterogeneity_analysis_summary.ipynb`
